In [2]:
import pandas as pd
import plotly.express as px

mall = pd.read_csv('../data/v1/mall.csv')
cost = pd.read_csv('../data/v1/cost.csv')
traffic_block_level = pd.read_csv('../data/v1/people_flow.csv')
store_to_external_mall = pd.read_csv('../data/v1/referenciel_traffic.csv')
traffic_mall_level = pd.read_csv('../data/v1/mall_traffic_data.csv')
store = pd.read_csv('../data/v1/store.csv')
dwell_time = pd.read_csv('../data/v1/dwell_time.csv')
# mall_name_mapping = pd.read_json('../data/v1/mapping_mall.json', typ='series') not useful



mall.set_index('mall_id', inplace=True)
mall['is_third_party'] = mall['mall_country'].str.contains('Third Parties').astype(int)

import numpy as np
cost["total_cost_r12m"] = np.sum(
    np.array([cost[i].fillna(0).values for i in cost.columns if i.endswith('r12m')])
, axis=0) - 2 * cost['rent_incentive_r12m']


store.drop(columns=[
    # 'store_year_closed',
    # 'store_year_opened',
    'store_name',
    
    ], inplace=True)
store = store.join(mall, on='mall_id')
store = pd.merge(store, cost, on='store_id', how='left')

store_to_external_mall.dropna(inplace=True)
store_to_external_mall.store_id = store_to_external_mall.store_id.astype(int)
store_to_external_mall.block_id = store_to_external_mall.block_id.astype(int)
store_to_external_mall.external_mall_id = store_to_external_mall.external_mall_id.astype(int)
store_to_external_mall.set_index('store_id', inplace=True)

store_index = store[["store_id", "mall_id"]]
mall_to_external_mall = store_index.join(store_to_external_mall, on='store_id', how='left')[['mall_id', 'external_mall_id']]
mall_to_external_mall = mall_to_external_mall.dropna().drop_duplicates()

traffic_block_level = traffic_block_level[traffic_block_level['days_with_traffic_data'] >= 5]


In [3]:
cost

,store_id,rent_incentive_r12m,service_charges_r12m,marketing_costs_r12m,administration_fees_r12m,real_estate_taxes_r12m,large_scale_works_r12m,maintenance_works_r12m,total_cost_r12m
0,0.0,NaN,16197.0,10101.0,NaN,NaN,NaN,NaN,NaN
1,55.0,NaN,NaN,14420.0,NaN,NaN,652.0,NaN,NaN
2,259.0,NaN,NaN,2424.0,NaN,NaN,3481.0,44779.0,NaN
3,226.0,NaN,511.0,47797.0,NaN,3452.0,NaN,NaN,NaN
4,104.0,NaN,6509.0,2469.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
14650,15517.0,NaN,15580.0,0.0,NaN,NaN,NaN,NaN,NaN
14651,15505.0,NaN,NaN,0.0,7947.0,NaN,NaN,NaN,NaN
14652,15506.0,NaN,NaN,0.0,NaN,9598.0,NaN,NaN,NaN
14653,15540.0,NaN,73503.0,0.0,NaN,NaN,NaN,NaN,NaN


### QUESTIONS
- What are main drivers of rent?



## Malls

In [5]:
import plotly.express as px

# Aggregate sales data by country
sales_per_country = store.groupby('mall_country').agg({'store_sales_r12m': 'sum', 'is_third_party': 'max'}).reset_index()
sales_per_country.columns = ['Country', 'Total Sales', 'is_third_party']

# Sort the dataframe in descending order by total sales
sales_per_country = sales_per_country.sort_values(by='Total Sales', ascending=False)

# Plot the data
fig = px.bar(sales_per_country, x='Country', y='Total Sales', title='Total Sales per Country', color='is_third_party')
fig.update_layout(xaxis_title='Country', yaxis_title='Total Sales', xaxis_tickangle=-45)
fig.update_layout(coloraxis_showscale=False)
fig.update_traces(marker=dict(color=['#636EFA' if x == 0 else '#EF553B' for x in sales_per_country['is_third_party']]))
fig.update_layout(
    height=600,
    width=800,
)
fig.show()

In [26]:
# Define custom colors for specific countries
color_map = {
    'France': '#00CC96',
    'USA': '#00CC96',
    'Germany': '#00CC96'
}

# Set default color to grey for all other countries
default_color = '#B0B0B0'

# Apply the custom colors to the box plot
fig = px.box(mall, x='mall_country', y='total_mall_area', title='Mall Size per Country', color='mall_country', color_discrete_map=color_map)
fig.for_each_trace(lambda t: t.update(marker_color=color_map.get(t.name, default_color)))
fig.update_layout(xaxis_title='Country', yaxis_title='Mall Area (m²)', xaxis_tickangle=-45, showlegend=False)
fig.show()

## Stores

In [27]:
store_index = store[["store_id", "mall_id"]]
mall_to_external_mall = store_index.join(store_to_external_mall, on='store_id', how='left')[['mall_id', 'external_mall_id']]
mall_to_external_mall = mall_to_external_mall.dropna().drop_duplicates()
mall_to_external_mall 

duplicate_external_mall_ids = mall_to_external_mall[mall_to_external_mall['external_mall_id'].duplicated(keep=False)]
# by setting keep=False we get all duplicates
print(duplicate_external_mall_ids)

       mall_id  external_mall_id
4069        27              32.0
12069       68              32.0


In [28]:
missing_values = store.isna().sum() / store.shape[0]
nonzero_missing_values = missing_values[missing_values > 0]
pd.DataFrame(nonzero_missing_values).T

,store_year_closed,store_sales_r12m,store_rent_r12m,rent_incentive_r12m,service_charges_r12m,marketing_costs_r12m,administration_fees_r12m,real_estate_taxes_r12m,large_scale_works_r12m,maintenance_works_r12m,total_cost_r12m
0,0.702077,0.242364,0.271301,0.983281,0.33194,0.093242,0.825928,0.577133,0.855572,0.714616,0.983281


Drop stores that lack rent/sales info

In [29]:
store.dropna(subset=['store_sales_r12m', 'store_sales_r12m'], inplace=True)

In [30]:
import plotly.express as px

# Count the number of closed stores
closed_stores_count = store['store_year_closed'].notna().sum()
open_stores_count = store['store_year_closed'].isna().sum()

# Create a dataframe for the pie chart
store_status = pd.DataFrame({
    'Status': ['Closed', 'Open'],
    'Count': [closed_stores_count, open_stores_count]
})

# Plot the pie chart
fig = px.pie(store_status, values='Count', names='Status', title='Open stores at capture')
fig.show()

We focus on stores that are open today, to have a uniform snapshot of sales and rent expenses.

In [31]:
store = store[store['store_year_closed'].isna()]
store.drop(columns=['store_year_closed'], inplace=True)

In [32]:
import plotly.express as px

# Count the number of stores per mall
stores_per_mall = store['mall_name'].value_counts().reset_index()
stores_per_mall.columns = ['Mall Name', 'Number of Stores']


# Plot histogram of the number of stores per mall
fig = px.box(stores_per_mall, x='Number of Stores', title='Histogram of Number of Stores per Mall')
fig.update_layout(xaxis_title='Number of Stores', yaxis_title='Count')
# Calculate the median number of stores per mall
median_stores_per_mall = stores_per_mall['Number of Stores'].median()

# Annotate the median on the plot
fig.add_annotation(
    x=median_stores_per_mall,
    y=0.5,
    text=f"Median: {median_stores_per_mall}",
    showarrow=True,
    arrowhead=2,
    ax=0,
    ay=-40
)
fig.show()

In [33]:
import plotly.express as px

# Count the number of stores per store_branch_level_1
store_branch_distribution = store['store_branch_level_1'].value_counts().reset_index()
store_branch_distribution.columns = ['store_branch_level_1', 'count']

# Plot the data
fig = px.bar(store_branch_distribution, x='store_branch_level_1', y='count', title='Distribution of Store Categories')
fig.update_layout(xaxis_title='Store Branch Level 1', yaxis_title='Count', xaxis_tickangle=-45)
fig.show()

In [34]:
# Calculate average rent per store_branch_level_1
average_rent = store.groupby('store_branch_level_1')['store_rent_r12m'].mean().dropna().reset_index()

# Sort the results by store_rent_r12m in descending order
average_rent = average_rent.sort_values(by='store_rent_r12m', ascending=False)

# Plot the data
fig = px.bar(average_rent, x='store_branch_level_1', y='store_rent_r12m', title='Average Rent by Store Branch Level 1')
fig.update_layout(xaxis_title='Store Branch Level 1', yaxis_title='Average Rent (R12M)', xaxis_tickangle=-45)
fig.show()

Substitute 0 with NA to get realistic results.

In [35]:
store.replace({'store_area': {0: None}}, inplace=True)

In [36]:
# Remove NAs in store_area
store_cleaned = store.dropna(subset=['store_area'])

# Plot the store area distribution by store_branch_level_1
fig = px.box(store_cleaned, x='store_branch_level_1', y='store_area', title='Store Area Distribution by Category')
fig.update_layout(xaxis_title='Store Type', yaxis_title='Store Area (m²)', xaxis_tickangle=-45)
fig.show()

In [37]:
import plotly.express as px

# Plot rent against store area
fig = px.scatter(store_cleaned, x='store_area', y='store_rent_r12m', title='Rent vs Store Area', log_y=False)
fig.update_layout(xaxis_title='Store Area (m²)', yaxis_title='Yearly rent')
fig.show()

Not much correlation between store size and rent

In [38]:
store = pd.merge(store, store_to_external_mall, on='store_id', how='left')
store = pd.merge(store, dwell_time, on=['block_id', 'external_mall_id'], how='left')

In [39]:
import plotly.express as px

# Group by block_id and calculate the mean of average_dwell_time and store_sales_r12m
block = store.groupby('block_id').agg({'average_dwell_time': 'mean', 'store_sales_r12m': 'mean'}).reset_index()
# Plot average dwell time against sales
fig = px.scatter(block, x='average_dwell_time', y='store_sales_r12m', title='Dwell time in block vs Sales')
fig.update_layout(xaxis_title='Average Dwell Time (minutes)', yaxis_title='Sales (log scale, averaged over block)')
fig.show()

In [40]:
import plotly.express as px

# Plot average dwell time vs median dwell time
fig = px.scatter(dwell_time, x='average_dwell_time', y='median_dwell_time', title='Average vs Median Dwell Time')
fig.update_layout(xaxis_title='Average Dwell Time (minutes)', yaxis_title='Median Dwell Time (minutes)')

# Add a line y=x in red
fig.add_shape(
    type='line',
    x0=dwell_time['average_dwell_time'].min(),
    y0=dwell_time['average_dwell_time'].min(),
    x1=dwell_time['average_dwell_time'].max(),
    y1=dwell_time['average_dwell_time'].max(),
    line=dict(color='Red', dash='dash')
)
fig.show()

Average dwell time is consistently greater than the median, suggesting that the distribution of dwell-time is right-skewed, i.e. few long-stayers and a bulk of people spending a shorter time in the block/mall

In [41]:
block = store.groupby('block_id').agg({
    'store_sales_r12m': 'sum',
    'average_dwell_time': 'mean',
    'store_area': 'sum',
    'store_rent_r12m': 'sum'
})

block = pd.merge(block, traffic_block_level, on='block_id', how='left')

In [42]:
import plotly.express as px

# Plot store visits against sales
fig = px.scatter(block, x='store_visits_monthly_average', y='store_rent_r12m', title='Store Visits vs Sales')
fig.update_layout(xaxis_title='Store Visits Monthly Average', yaxis_title='Avg Rent in block')
fig.show()

## France Focus

In [43]:
store_france = store.copy()[store['mall_country'] == 'France']

In [44]:
mall_france = store_france.groupby('mall_id').agg({'store_sales_r12m': 'sum', 'store_rent_r12m': 'sum'}).reset_index()
mall_france = pd.merge(mall_france, mall_to_external_mall, on='mall_id', how='left')
mall_france = pd.merge(mall_france, mall, on='mall_id', how='left')
# mall_france = pd.merge(mall_france, traffic_mall_level, on='external_mall_id', how='left')
# mall_france.dropna(subset=['external_mall_id'], inplace=True)

mall_france['sales_per_sqm'] = mall_france['store_sales_r12m'] / mall_france['total_mall_area']
mall_france['rent_per_sqm'] = mall_france['store_rent_r12m'] / mall_france['total_mall_area']


mall_france.sort_values(by='sales_per_sqm', ascending=False, inplace=True)
best_malls_sales = mall_france.copy().head(10)
worst_malls_sales = mall_france.copy().tail(10)

In [45]:
import plotly.express as px

# Filter stores in best and worst malls
best_malls_sales_stores = store[store['mall_id'].isin(best_malls_sales['mall_id'])]
worst_malls_sales_stores = store[store['mall_id'].isin(worst_malls_sales['mall_id'])]

# Count the occurrences of each store_branch_level_1
best_malls_sales_branch_count = best_malls_sales_stores['store_branch_level_1'].value_counts(normalize=True).reset_index()
best_malls_sales_branch_count.columns = ['store_branch_level_1', 'count']

worst_malls_sales_branch_count = worst_malls_sales_stores['store_branch_level_1'].value_counts(normalize=True).reset_index()
worst_malls_sales_branch_count.columns = ['store_branch_level_1', 'count']

# Keep only the first 5 categories
best_malls_sales_branch_count = best_malls_sales_branch_count.head(7)
worst_malls_sales_branch_count = worst_malls_sales_branch_count[worst_malls_sales_branch_count['store_branch_level_1'].isin(best_malls_sales_branch_count['store_branch_level_1'])]

# Add a column to distinguish between best and worst malls
best_malls_sales_branch_count['Category'] = 'Best Malls'
worst_malls_sales_branch_count['Category'] = 'Worst Malls'

# Combine the dataframes
combined_branch_count = pd.concat([best_malls_sales_branch_count, worst_malls_sales_branch_count])

# Plot the data
fig = px.bar(combined_branch_count, x='store_branch_level_1', y='count', color='Category', barmode='group', title='Best vs Worst Malls by Sales per sqm')
fig.update_layout(xaxis_title='Store category', yaxis_title='Fraction of stores', xaxis_tickangle=-45)
fig.update_layout(yaxis_tickformat='.0%')
fig.show()


In [46]:
import plotly.express as px

# Combine the best and worst mall sales data
best_malls_sales['Category'] = 'Best Malls'
worst_malls_sales['Category'] = 'Worst Malls'
combined_mall_sales = pd.concat([best_malls_sales, worst_malls_sales])

# Plot the data
fig = px.box(combined_mall_sales, x='Category', y='rent_per_sqm', title='Comparison of Rent per sqm between Best and Worst Malls')
fig.update_layout(xaxis_title='Category', yaxis_title='Rent per sqm')
fig.show()

Stores with highes sales also have highest rent per sqm. This makes sense but what is the causality? Is rent high because its an appealing location?

In [236]:
store_france['sales_per_sqm'] = store_france['store_sales_r12m'] / store_france['store_area']
store_france['rent_per_sqm'] = store_france['store_rent_r12m'] / store_france['store_area']

In [237]:
# Convert sales_per_sqm and rent_per_sqm to numeric, forcing errors to NaN
store_france['sales_per_sqm'] = pd.to_numeric(store_france['sales_per_sqm'], errors='coerce')
store_france['rent_per_sqm'] = pd.to_numeric(store_france['rent_per_sqm'], errors='coerce')

# Group by store_branch_level_1 and calculate the mean of sales_per_sqm and rent_per_sqm
category_sales_rent = store_france.groupby('store_branch_level_1').agg({
    'sales_per_sqm': 'mean',
    'rent_per_sqm': 'mean'
}).dropna().reset_index()

# Sort by sales_per_sqm in descending order
highest_sales_per_sqm = category_sales_rent.sort_values(by='sales_per_sqm', ascending=False).head(10)

# Sort by rent_per_sqm in descending order
highest_rent_per_sqm = category_sales_rent.sort_values(by='rent_per_sqm', ascending=False).head(10)

In [238]:
highest_sales_per_sqm

,store_branch_level_1,sales_per_sqm,rent_per_sqm
13,Services,90230.854028,15096.005895
11,Jewellery,63184.626762,12502.001795
0,Bags & Footwear & Accessories,59324.441022,9288.654941
6,Food & Beverage Services,52645.498829,7608.822172
9,Health & Beauty,44691.700924,6543.105363
8,Gifts,42308.557337,6042.529913
1,Culture & Media & Technology,30200.848386,3974.564665
12,Other,30091.005308,8890.992292
4,Fashion apparel,28686.059430,3214.535474
14,Sport,20654.747678,2031.394415


In [239]:
highest_rent_per_sqm

,store_branch_level_1,sales_per_sqm,rent_per_sqm
13,Services,90230.854028,15096.005895
11,Jewellery,63184.626762,12502.001795
3,Entertainment,19477.764466,11624.591193
0,Bags & Footwear & Accessories,59324.441022,9288.654941
12,Other,30091.005308,8890.992292
6,Food & Beverage Services,52645.498829,7608.822172
9,Health & Beauty,44691.700924,6543.105363
8,Gifts,42308.557337,6042.529913
1,Culture & Media & Technology,30200.848386,3974.564665
4,Fashion apparel,28686.059430,3214.535474


Factors to consider: 
- A: how much money the individual store brings
- B: how much store presence benefits other stores ?

In [240]:
store["margin_r12m_1pc"] = store["store_sales_r12m"] * 0.01 + store["store_rent_r12m"] - store["total_cost_r12m"]
store["margin_r12m_2pc"] = store["store_sales_r12m"] * 0.02 + store["store_rent_r12m"] - store["total_cost_r12m"]
store["margin_r12m_3pc"] = store["store_sales_r12m"] * 0.03 + store["store_rent_r12m"] - store["total_cost_r12m"]
store["margin_r12m_4pc"] = store["store_sales_r12m"] * 0.04 + store["store_rent_r12m"] - store["total_cost_r12m"]
store["margin_r12m_5pc"] = store["store_sales_r12m"] * 0.05 + store["store_rent_r12m"] - store["total_cost_r12m"]

store["margin_r12m_1pc_per_sqm"] = store["margin_r12m_1pc"] / store["store_area"]
store["margin_r12m_2pc_per_sqm"] = store["margin_r12m_2pc"] / store["store_area"]
store["margin_r12m_3pc_per_sqm"] = store["margin_r12m_3pc"] / store["store_area"]
store["margin_r12m_4pc_per_sqm"] = store["margin_r12m_4pc"] / store["store_area"]
store["margin_r12m_5pc_per_sqm"] = store["margin_r12m_5pc"] / store["store_area"]

In [241]:
import plotly.express as px

# Prepare the data for plotting
margins_data = store[['margin_r12m_1pc_per_sqm', 'margin_r12m_2pc_per_sqm', 'margin_r12m_3pc_per_sqm', 'margin_r12m_4pc_per_sqm', 'margin_r12m_5pc_per_sqm']].melt(var_name='Commission Level', value_name='Margin per sqm')

# Rename the commission levels for better readability
margins_data['Commission Level'] = margins_data['Commission Level'].replace({
    'margin_r12m_1pc_per_sqm': '1% Commission',
    'margin_r12m_2pc_per_sqm': '2% Commission',
    'margin_r12m_3pc_per_sqm': '3% Commission',
    'margin_r12m_4pc_per_sqm': '4% Commission',
    'margin_r12m_5pc_per_sqm': '5% Commission'
})

# Remove outliers using IQR method
Q1 = margins_data['Margin per sqm'].quantile(0.25)
Q3 = margins_data['Margin per sqm'].quantile(0.75)
IQR = Q3 - Q1
filtered_margins_data = margins_data[~((margins_data['Margin per sqm'] < (Q1 - 1.5 * IQR)) | (margins_data['Margin per sqm'] > (Q3 + 1.5 * IQR)))]

# Plot the boxplots
fig = px.box(filtered_margins_data, x='Commission Level', y='Margin per sqm', title='Margins per sqm at Increasing Levels of Sales Commission')
fig.update_layout(xaxis_title='Commission Level', yaxis_title='Margin per sqm')
fig.show()